<a href="https://colab.research.google.com/github/Kadorath/NLPHW1/blob/main/finetune.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
# Use this only after you check everything is being loaded properly

# First install necessary libraries
# Exclamation marks for shell commands
! pip install -U transformers datasets evaluate scikit-learn
! pip install accelerate -U
! pip install tqdm math time
! pip install wandb

In [ ]:
import torch

device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
print(f'device: {device}')

device: cuda:0


In [ ]:
from datasets import load_dataset

hs_ds = load_dataset("hate_speech_offensive")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/24783 [00:00<?, ? examples/s]

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')

text = 'Hello everyone! ! antidisestablishmentarianism'

print(f'\n\n{tokenizer.name_or_path}')
vocab = {v: k for k, v in tokenizer.vocab.items()}
tokenized_text = tokenizer(text)
print([vocab[id] for id in tokenized_text['input_ids']])

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]



distilbert-base-uncased
['[CLS]', 'hello', 'everyone', '!', '!', 'anti', '##dis', '##est', '##ab', '##lish', '##ment', '##arian', '##ism', '[SEP]']


In [ ]:
hs_ds['train'] = hs_ds['train'].rename_column('class', 'label')
hs_ds = hs_ds['train'].train_test_split(test_size=0.1)
print(hs_ds)

DatasetDict({
    train: Dataset({
        features: ['count', 'hate_speech_count', 'offensive_language_count', 'neither_count', 'label', 'tweet'],
        num_rows: 22304
    })
    test: Dataset({
        features: ['count', 'hate_speech_count', 'offensive_language_count', 'neither_count', 'label', 'tweet'],
        num_rows: 2479
    })
})


In [ ]:
hs_ds['train']['tweet'][1]

'RT @LamborghiniBow: Fuck a hoe, respect a woman, love a lady&#128175;'

In [ ]:
def preprocess_function(examples):
    return tokenizer(examples['tweet'], truncation=True)

In [ ]:
tokenized_hs = hs_ds.map(preprocess_function, batched=True)

Map:   0%|          | 0/22304 [00:00<?, ? examples/s]

Map:   0%|          | 0/2479 [00:00<?, ? examples/s]

In [ ]:
print(tokenized_hs)

DatasetDict({
    train: Dataset({
        features: ['count', 'hate_speech_count', 'offensive_language_count', 'neither_count', 'label', 'tweet', 'input_ids', 'attention_mask'],
        num_rows: 22304
    })
    test: Dataset({
        features: ['count', 'hate_speech_count', 'offensive_language_count', 'neither_count', 'label', 'tweet', 'input_ids', 'attention_mask'],
        num_rows: 2479
    })
})


In [ ]:
vocab = {v: k for k, v in tokenizer.vocab.items()}
tokenized_text = tokenized_hs['train'][12]
print([vocab[id] for id in tokenized_text['input_ids']])

['[CLS]', '@', 'cute', '##eme', '##rgen', '##cy', 'shown', '*', 'a', 'tiny', 'kitten', 'you', 'dumb', 'fucking', 'cu', '##nt', '[SEP]']


In [ ]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
import evaluate

# Proportion of correct predictions among the total number of cases processed
accuracy = evaluate.load('accuracy')

In [ ]:
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [ ]:
labels = ['HATE-SPEECH', 'OFFENSIVE-LANGUAGE', 'NEITHER']
id2label = {i: label for i, label in enumerate(labels)}
label2id = {label: i for i, label in id2label.items()}

print('id2label:', id2label)
print('label2id:', label2id)

id2label: {0: 'HATE-SPEECH', 1: 'OFFENSIVE-LANGUAGE', 2: 'NEITHER'}
label2id: {'HATE-SPEECH': 0, 'OFFENSIVE-LANGUAGE': 1, 'NEITHER': 2}


In [ ]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

# This automodel class gives us the model with pretrained weights + a sequence classification head
# We specify how many labels we need so that the model has the correct number of outputs
# We specify id2label/label2id so that the model understands the label associated with each output
model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased", num_labels=len(labels), id2label=id2label, label2id=label2id
)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
import wandb
wandb.login()


<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
run = wandb.init(
    # Set the project where this run will be logged
    project="my-awesome-project",
    # Track hyperparameters and run metadata
    config={
        "learning_rate": 2e-5,
        "epochs": 3,
    },
)

wandb: Currently logged in as: wong0683 (wong0683umn). Use `wandb login --relogin` to force relogin


In [ ]:
import time
import math
from tqdm import tqdm



class CustomTrainer(Trainer):
  def _inner_training_loop(self,batch_size=None,args=None,
                           resume_from_checkpoint=None,trial=None,
                           ignore_keys_for_eval=None
  ):
    number_of_epochs = args.num_train_epochs
    start = time.time()
    train_loss=[]
    train_acc=[]
    eval_acc=[]

    criterion = torch.nn.CrossEntropyLoss().to(device)
    self.optimizer = torch.optim.Adam(model.parameters(), lr=args.learning_rate)
    self.scheduler = torch.optim.lr_scheduler.StepLR(self.optimizer,1,gamma=0.9)

    train_dataloader = self.get_train_dataloader()
    eval_dataloader = self.get_eval_dataloader()

    max_steps = math.ceil(args.num_train_epochs * len(train_dataloader))
    for epoch in range(number_of_epochs):
      train_loss_per_epoch = 0
      train_acc_per_epoch = 0
      with tqdm(train_dataloader, unit="batch") as training_epoch:
        training_epoch.set_description(f"Training Epoch {epoch}")

        for step, inputs in enumerate(training_epoch):
          inputs = inputs.to(device)
          labels = inputs["labels"]
          output = model(inputs["input_ids"])
          loss = criterion(output["logits"], labels)
          train_loss_per_epoch += loss.item()

          self.optimizer.zero_grad()
          loss.backward()
          self.optimizer.step()
          acc = (output['logits'].argmax(dim=1)==labels).sum().item()
          train_acc_per_epoch += (output['logits'].argmax(dim=1)==labels).sum().item()
          wandb.log({"train-loss": loss.item(), "train-acc": acc})

      self.scheduler.step()
      train_loss_per_epoch /= len(train_dataloader)
      train_acc_per_epoch /= (len(train_dataloader)*batch_size)

      eval_loss_per_epoch = 0
      eval_acc_per_epoch = 0
      with tqdm(eval_dataloader,unit="batch") as eval_epoch:
        eval_epoch.set_description(f"Evaluation Epoch {epoch}")
        for step, inputs in enumerate(eval_epoch):
          inputs = inputs.to(device)
          labels = inputs["labels"]

          output = model(inputs["input_ids"])
          loss = criterion(output["logits"], labels)
          eval_loss_per_epoch += loss.item()
          acc = (output['logits'].argmax(dim=1)==labels).sum().item()
          eval_acc_per_epoch += acc
          wandb.log({"eval-loss": loss.item(), "eval-acc": acc})


      eval_loss_per_epoch /= (len(eval_dataloader))
      eval_acc_per_epoch /= (len(eval_dataloader)*batch_size)

      print(f"\tTrain Loss: {train_loss_per_epoch:.3f} | Train Acc: {train_acc_per_epoch*100:.2f}%")
      print(f"\tEval Loss: {eval_loss_per_epoch:.3f} | Eval Acc: {eval_acc_per_epoch*100:.2f}%")
      wandb.log({"train-acc": train_acc_per_epoch, "train-loss": train_loss_per_epoch, "eval-acc": eval_acc_per_epoch, "eval-loss": eval_loss_per_epoch})

    print(f'Time: {(time.time()-start)/60:.3f} minutes')

In [ ]:
# https://huggingface.co/transformers/v4.4.2/main_classes/trainer.html#trainingarguments
training_args = TrainingArguments(
    output_dir='my_awesome_model',
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=3,
    weight_decay=0.01,
    evaluation_strategy='epoch',
    save_strategy='epoch',
    load_best_model_at_end=True,
)

In [ ]:
# https://huggingface.co/transformers/v4.4.2/main_classes/trainer.html#id1
trainer = CustomTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_hs['train'],
    eval_dataset=tokenized_hs['test'],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()  # do not do this in HW1 :) you will

Training Epoch 0:  48%|████▊     | 332/697 [01:08<01:12,  5.02batch/s]

In [ ]:
from transformers import pipeline

classifier = pipeline('sentiment-analysis', model=model, tokenizer=tokenizer, device=device)

In [ ]:
hate_speech = []
offense_speech = []
neutral_speech = []
for entry in (hs_ds['train']):
  match(entry['label']):
    case 0:
      hate_speech.append(entry)
    case 1:
      offense_speech.append(entry)
    case 2:
      neutral_speech.append(entry)

torch.save(model.state_dict(), PATH)

model = TheModelClass(*args, **kwargs)
model.load_state_dict(torch.load(PATH))
model.eval()

In [ ]:
text = hate_speech[410]
print(text)
print(str(text["tweet"]) + ": " + str(text['label']))
classifier(text["tweet"])

{'count': 3, 'hate_speech_count': 2, 'offensive_language_count': 1, 'neither_count': 0, 'label': 0, 'tweet': '@amandabynes i can give you some of my clothes so you can look like a cute dyke'}
@amandabynes i can give you some of my clothes so you can look like a cute dyke: 0


[{'label': 'HATE-SPEECH', 'score': 0.5040497779846191}]

In [ ]:
matrix = [[0,0,0],[0,0,0],[0,0,0]]
for entry in hs_ds['test']:
  prediction = label2id[classifier(entry['tweet'])[0]['label']]
  matrix[entry['label']][prediction] += 1


/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1123: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


In [ ]:
import copy

print(matrix)
percent_matrix = copy.deepcopy(matrix)
sums = [sum(matrix[0]),sum(matrix[1]),sum(matrix[2])]
for row in range(3):
  for col in range(3):
    percent_matrix[row][col] /= sums[row]

print(sums)
print(percent_matrix)

[[68, 69, 8], [73, 1795, 32], [11, 58, 365]]
[145, 1900, 434]
[[0.4689655172413793, 0.47586206896551725, 0.05517241379310345], [0.038421052631578946, 0.9447368421052632, 0.016842105263157894], [0.02534562211981567, 0.1336405529953917, 0.8410138248847926]]


In [ ]:
!zip -r /content/hs_default_model.zip /content/my_awesome_model

  adding: content/my_awesome_model/ (stored 0%)
  adding: content/my_awesome_model/runs/ (stored 0%)
  adding: content/my_awesome_model/runs/Feb07_23-39-26_326f7f81d58d/ (stored 0%)
  adding: content/my_awesome_model/runs/Feb07_23-39-26_326f7f81d58d/events.out.tfevents.1707349166.326f7f81d58d.607.0 (deflated 57%)
  adding: content/my_awesome_model/checkpoint-1394/ (stored 0%)
  adding: content/my_awesome_model/checkpoint-1394/config.json (deflated 46%)
  adding: content/my_awesome_model/checkpoint-1394/trainer_state.json (deflated 60%)
  adding: content/my_awesome_model/checkpoint-1394/special_tokens_map.json (deflated 42%)
  adding: content/my_awesome_model/checkpoint-1394/optimizer.pt (deflated 24%)
  adding: content/my_awesome_model/checkpoint-1394/scheduler.pt (deflated 56%)
  adding: content/my_awesome_model/checkpoint-1394/training_args.bin (deflated 51%)
  adding: content/my_awesome_model/checkpoint-1394/model.safetensors (deflated 8%)
  adding: content/my_awesome_model/checkpoi

In [ ]:
! du -sh /content/my_awesome_model/

1.5G	/content/my_awesome_model/
